Импорт необходимых инструментов

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC

pd.set_option('display.max_columns', None)

Считывание данных в Data Frame и вывод первых 5 строк

In [2]:
titanic = pd.read_csv('/Users/alekseyfomin/Documents/Kaggle/titanic_train.csv')

titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Просмотр общей информации 

In [3]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Удалим некоторые неинформативные признаки (ОПЦИОНАЛЬНО, НЕКОТОРЫЕ ПРИЗНАКИ ИЗ УДАЛЕННЫХ МОГЛИ БЫТЬ ПОЛЕЗНЫ)

In [4]:
titanic.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)

Расчет основных статистик

In [5]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Переименуем целевую переменную и переместим ее на место последней колонки

In [6]:
titanic.rename(columns={'Survived': 'Target'}, inplace=True)
titanic.head(1)

,PassengerId,Target,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.25,S


In [7]:
target_col = titanic.pop('Target')

titanic.insert(8, 'Target', target_col)


In [8]:
titanic.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Target
0,1,3,male,22.0,1,0,7.2500,S,0
1,2,1,female,38.0,1,0,71.2833,C,1
2,3,3,female,26.0,0,0,7.9250,S,1
3,4,1,female,35.0,1,0,53.1000,S,1
4,5,3,male,35.0,0,0,8.0500,S,0


Проверяем данные на дубликаты

In [9]:
titanic.duplicated().sum()

0

Определение пропущенных значений

In [10]:
titanic.isna().sum()

PassengerId      0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
Target           0
dtype: int64

Размер таблицы до удаления пропущенных значений

In [11]:
titanic.shape

(891, 9)

После

In [12]:
titanic.dropna(subset=['Embarked'], inplace=True)

titanic.shape

(889, 9)

Видим, что удалили две строки из таблицы

Далее необходимо обработать остальные колонки, в которыйх есть пропущенные значения

In [14]:
titanic.Age.describe()

count    712.000000
mean      29.642093
std       14.492933
min        0.420000
25%       20.000000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

Заменим пропущенные значения возраста на среднее значение по таблице

In [15]:
titanic['Age'].fillna(round(titanic.Age.mean()), inplace=True)

Проверим наши данные

In [17]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Pclass       889 non-null    int64  
 2   Sex          889 non-null    object 
 3   Age          889 non-null    float64
 4   SibSp        889 non-null    int64  
 5   Parch        889 non-null    int64  
 6   Fare         889 non-null    float64
 7   Embarked     889 non-null    object 
 8   Target       889 non-null    int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 69.5+ KB


Отлично, мы убедились, что нет дубликатов, пропущенных значений, но имеем два нечисловых признака, поработаем с ними

In [21]:
titanic.select_dtypes(include='object')

,Sex,Embarked
0,male,S
1,female,C
2,female,S
3,female,S
4,male,S
...,...,...
886,male,S
887,female,S
888,female,S
889,male,C


In [22]:
titanic.Sex.value_counts()

male      577
female    312
Name: Sex, dtype: int64

In [34]:
titanic.Sex = pd.Categorical(titanic.Sex)

In [35]:
titanic.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [36]:
titanic.Embarked = pd.Categorical(titanic.Embarked)

In [51]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  889 non-null    int64   
 1   Pclass       889 non-null    int64   
 2   Sex          889 non-null    category
 3   Age          889 non-null    float64 
 4   SibSp        889 non-null    int64   
 5   Parch        889 non-null    int64   
 6   Fare         889 non-null    float64 
 7   Embarked     889 non-null    category
 8   Target       889 non-null    int64   
dtypes: category(2), float64(2), int64(5)
memory usage: 57.5 KB


In [52]:
titanic = pd.get_dummies(titanic)

Мы сделали минимально-необходимую предварительную обработку данных для подачи данных алгоритму для обучения модели, однако осталось проверить сбалансированность целевой переменной для определения подходящей метрики (если метрика не была указана заранее заказчиком)

In [54]:
titanic.Target.value_counts()

0    549
1    340
Name: Target, dtype: int64

Видим, что соотношение классов в таблице +- 60/40, в таком случае можно использовать метрику **Accuracy**, так как классы более-менее сбалансированные

Разделим наш датасет на тренировочную и тестовую выборки

In [55]:
X = titanic.drop('Target', axis=1)
y = titanic.Target

In [56]:
X

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,3,35.0,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,27.0,0,0,13.0000,0,1,0,0,1
887,888,1,19.0,0,0,30.0000,1,0,0,0,1
888,889,3,30.0,1,2,23.4500,1,0,0,0,1
889,890,1,26.0,0,0,30.0000,0,1,1,0,0


In [57]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Target, Length: 889, dtype: int64

In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

Обучим нашу модель на алгоритме ***sklearn.svm.SVC***

In [59]:
svc = SVC()
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.6629213483146067

Мы показали вам примитивный подход.\
Как правило, далее, специалисты в сфере data science, если результаты не устраивают заказчика или есть идеи улучшения качества модели(-ей), работают с данными, генерируют новые признаки, обрабатывают уже имеющиеся, перебирают гиперпараметры модели (или, что практикуется КРАЙНЕ РЕДКО, разрабатывают собственные алгоритмы для решения задачи)